# 手动标注数千条文本的质量，用于冷启动

In [1]:
import os, sys, json, random
from PyQt6.QtWidgets import QApplication, QWidget, QVBoxLayout, QHBoxLayout, QPushButton, QTextEdit

In [2]:
def get_rand_text():
    fnames = os.listdir('WuDaoCorpus2.0_base_200G')
    fname = fnames[random.randint(0,len(fnames)-1)]
    with open('WuDaoCorpus2.0_base_200G/'+fname,'r',encoding='utf-8') as f:
        temp = [item['content'] for item in json.load(f)]
        temp = random.sample(temp,len(temp)//10000)
    return temp
# with open('auto_mid.txt','r',encoding='utf-8') as f:
#     mid_data = [line[:-1] for line in f]
# def get_rand_text():
#     global mid_data
#     if len(mid_data) > 0:
#         ret = mid_data[:10]
#         mid_data = mid_data[10:]
#         return ret
#     else:
#         return ["文本标注完成"]*2

In [3]:
class TextLabelingWindow(QWidget):
    def __init__(self,file_high,file_mid,file_low):
        super().__init__()
        global rand_text_list
        # 设置窗口标题和大小
        self.setWindowTitle('人工文本标注')
        self.setFixedSize(800, 850)

        # 创建布局
        layout = QVBoxLayout()

        # 创建标签并设置显示文本
        self.text_edit = QTextEdit()
        self.text_edit.setText(rand_text_list[-1])  # 模拟长文本
        rand_text_list = rand_text_list[:-1]
        self.text_edit.setReadOnly(True)  # 设置为只读，防止用户修改文本
        layout.addWidget(self.text_edit)

        # 创建按钮布局（横向）
        button_layout = QHBoxLayout()

        # 创建按钮
        self.high_button = QPushButton('高')
        self.mid_button = QPushButton('中')
        self.low_button = QPushButton('低')

        # 连接按钮点击事件
        self.high_button.clicked.connect(self.mark_high)
        self.mid_button.clicked.connect(self.mark_mid)
        self.low_button.clicked.connect(self.mark_low)

        # 将按钮添加到横向布局
        button_layout.addWidget(self.high_button)
        button_layout.addWidget(self.mid_button)
        button_layout.addWidget(self.low_button)

        # 将横向布局添加到主布局
        layout.addLayout(button_layout)

        # 设置窗口的布局
        self.setLayout(layout)

    def mark_high(self):
        global rand_text_list
        print(self.text_edit.toPlainText(),file=file_high)
        self.text_edit.setText(rand_text_list[-1])
        rand_text_list = rand_text_list[:-1]
        if len(rand_text_list) == 0:
            rand_text_list = get_rand_text()

    def mark_mid(self):
        global rand_text_list
        print(self.text_edit.toPlainText(),file=file_mid)
        self.text_edit.setText(rand_text_list[-1])
        rand_text_list = rand_text_list[:-1]
        if len(rand_text_list) == 0:
            rand_text_list = get_rand_text()

    def mark_low(self):
        global rand_text_list
        print(self.text_edit.toPlainText(),file=file_low)
        self.text_edit.setText(rand_text_list[-1])
        rand_text_list = rand_text_list[:-1]
        if len(rand_text_list) == 0:
            rand_text_list = get_rand_text()

In [4]:
app = QApplication(sys.argv)
file_high = open('high_init.txt','a')
file_mid = open('mid_init.txt','a')
file_low = open('low_init.txt','a')
rand_text_list = get_rand_text()
window = TextLabelingWindow(file_high,file_mid,file_low)
window.show()
if app.exec() == 0:
    file_high.close()
    file_mid.close()
    file_low.close()
    print("Success!")

Success!


In [5]:
# with open('auto_mid.txt','w',encoding='utf-8') as f:
#     for line in mid_data:
#         print(line,file=f)